# Introduction

End-to-end machine learning project showing key aspects of developing and deploying real-life ML driven application. 

This includes,

* EDA, data manipulation an preparation
* Scraping additional features from external sources
* Iterative process of building ML model
* Wrapping it as Python module as transition from notebooks to production code
* Using this module in Flask based microservice
* Containerizing it with Docker, orchestring with Kubernetes and deploying on Google Cloud

This basically covers full ML tech stack at the moment (maybe besides online learning, but thats for other time).

We'll be using [SF Bay Area Bike Share](https://www.kaggle.com/benhamner/sf-bay-area-bike-share) dataset to model duration of bike travel across San Francisco. This project is all about tech stack and leveraging different tools and ML techniques to solve the problem.

## 1. Import dependencies

In [1]:
import os
import sqlite3
import json

from datetime import datetime

import pandas as pd

## 2. Define paths

In [2]:
input_data_path = os.path.join("..", "data", "input")
output_data_path = os.path.join("..", "data", "processed")

## 3. Database schema

In [3]:
con = sqlite3.connect(os.path.join(input_data_path, 'database.sqlite'))
cur = con.cursor()

In [4]:
cur.execute('SELECT name FROM sqlite_master')
tables = cur.fetchall()

In [5]:
tables

[('station',), ('status',), ('trip',), ('weather',)]

In [6]:
schemas = {}

for table in tables:
  cur.execute('select * from {} limit 1'.format(table[0]))
  schemas[table[0]] = [d[0] for d in cur.description]

In [7]:
schemas['station']

['id', 'name', 'lat', 'long', 'dock_count', 'city', 'installation_date']

In [8]:
schemas['trip']

['id',
 'duration',
 'start_date',
 'start_station_name',
 'start_station_id',
 'end_date',
 'end_station_name',
 'end_station_id',
 'bike_id',
 'subscription_type',
 'zip_code']

In [9]:
schemas['status']

['station_id', 'bikes_available', 'docks_available', 'time']

In [10]:
schemas['weather']

['date',
 'max_temperature_f',
 'mean_temperature_f',
 'min_temperature_f',
 'max_dew_point_f',
 'mean_dew_point_f',
 'min_dew_point_f',
 'max_humidity',
 'mean_humidity',
 'min_humidity',
 'max_sea_level_pressure_inches',
 'mean_sea_level_pressure_inches',
 'min_sea_level_pressure_inches',
 'max_visibility_miles',
 'mean_visibility_miles',
 'min_visibility_miles',
 'max_wind_Speed_mph',
 'mean_wind_speed_mph',
 'max_gust_speed_mph',
 'precipitation_inches',
 'cloud_cover',
 'events',
 'wind_dir_degrees',
 'zip_code']

## 4. Fetch altitude information

One of things I think missing is info about altitudes. It might bring additional information about bike travel times.

In [11]:
cur.execute('select count(distinct id) from station').fetchall()

[(70,)]

In [12]:
cur.execute('select distinct id, lat, long from station')
coords = cur.fetchall()

In [13]:
len(coords)

70

Now export and build scraper (will be in utils.py) and start EDA in another notebook !

In [14]:
payload = {}

for coord in coords:
  payload[coord[0]] = [coord[1], coord[2]]

In [15]:
with open(os.path.join(output_data_path, 'coordinates.json')  , 'w') as file:
  json.dump(payload, file)

## 5. Merge data

We've scraped altitude data using [Google Elevation API](https://developers.google.com/maps/documentation/elevation/intro). Now it's time to merge all data together. We will not be using 'status' table - it seems like redundand information for this task.

In [16]:
trips = pd.read_sql_query('select * from trip', con)
stations = pd.read_sql_query('select * from station', con)
weather = pd.read_sql_query('select * from weather', con)

### 5.1. Map altitudes to stations

In [17]:
stations.shape

(70, 7)

In [18]:
stations.head()

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,8/6/2013
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,8/5/2013
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,8/6/2013
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,8/5/2013
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,8/7/2013


In [19]:
alt = pd.read_json(os.path.join("../data/external", 'coordinate-altitudes.json') , orient='index')
alt.reset_index(inplace=True)
alt.columns = ['id', 'data', 'altitude',]

In [20]:
alt.head()

,id,data,altitude
0,2,"[37.329732, -121.90178200000001]",28.275824
1,3,"[37.330698, -121.888979]",28.038095
2,4,"[37.333988, -121.894902]",25.517740
3,5,"[37.331415, -121.8932]",26.192633
4,6,"[37.336721000000004, -121.894074]",25.543407


In [21]:
altstations = pd.merge(stations, alt, how='left', on='id')
altstations = altstations.drop(['data'], axis=1)

In [22]:
altstations.tail()

,id,name,lat,long,dock_count,city,installation_date,altitude
65,77,Market at Sansome,37.789625,-122.400811,27,San Francisco,8/25/2013,7.888969
66,80,Santa Clara County Civic Center,37.352601,-121.905733,15,San Jose,12/31/2013,18.939966
67,82,Broadway St at Battery St,37.798541,-122.400862,15,San Francisco,1/22/2014,6.239827
68,83,Mezes Park,37.491269,-122.236234,15,Redwood City,2/20/2014,3.690954
69,84,Ryland Park,37.342725,-121.895617,15,San Jose,4/9/2014,24.346684


### 5.2 Map stations to trips

Note to self - duration is in seconds !

In [23]:
trips.head()

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4069,174,8/29/2013 9:08,2nd at South Park,64,8/29/2013 9:11,2nd at South Park,64,288,Subscriber,94114
1,4073,1067,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:42,San Francisco Caltrain 2 (330 Townsend),69,321,Subscriber,94703
2,4074,1131,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,317,Subscriber,94115
3,4075,1117,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,316,Subscriber,94122
4,4076,1118,8/29/2013 9:25,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,322,Subscriber,94597


In [24]:
startstations = altstations[['id', 'lat', 'long', 'altitude']]
stopstations = altstations[['id', 'lat', 'long', 'altitude']]

In [25]:
startstations.columns = ['start_station_id', 'start_lat', 'start_long', 'start_alt']
stopstations.columns = ['end_station_id', 'end_lat', 'end_long', 'end_alt']

In [26]:
trips = pd.merge(trips, startstations, how='left', on='start_station_id')
trips = pd.merge(trips, stopstations, how='left', on='end_station_id')

### 5.3. Add some weather data

There are two keys - date and zip code

In [27]:
weather.head()

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
0,8/29/2013,74,68,61,61,58,56,93,75,57,...,10,10,23,11,28,0,4,,286,94107
1,8/30/2013,78,69,60,61,58,56,90,70,50,...,10,7,29,13,35,0,2,,291,94107
2,8/31/2013,71,64,57,57,56,54,93,75,57,...,10,10,26,15,31,0,4,,284,94107
3,9/1/2013,74,66,58,60,56,53,87,68,49,...,10,10,25,13,29,0,4,,284,94107
4,9/2/2013,75,69,62,61,60,58,93,77,61,...,10,6,23,12,30,0,6,,277,94107


In [28]:
trips['date'] = pd.to_datetime(trips['start_date'], format='%m/%d/%Y %H:%M')
trips['date'] = trips['date'].apply(lambda x: x.date())

In [29]:
weather['date'] = weather['date'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y').date())

In [30]:
df = pd.merge(trips, weather, how='left', on=['date', 'zip_code'])

In [31]:
df.head()

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,...,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees
0,4069,174,8/29/2013 9:08,2nd at South Park,64,8/29/2013 9:11,2nd at South Park,64,288,Subscriber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4073,1067,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:42,San Francisco Caltrain 2 (330 Townsend),69,321,Subscriber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4074,1131,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,317,Subscriber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4075,1117,8/29/2013 9:24,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,316,Subscriber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4076,1118,8/29/2013 9:25,South Van Ness at Market,66,8/29/2013 9:43,San Francisco Caltrain 2 (330 Townsend),69,322,Subscriber,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
df.to_csv(os.path.join(output_data_path, 'sfbike_full.csv'), index=False)

And that's all for this notebook !